# SysML Trace Execution

A notebook experimenting with execution rules and timeslices as cursors in SysML.

In [ ]:
import json
import pymbe.api as pm

import copy

from importlib import resources as lib_resources

from dataclasses import field
from typing import Any, List

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import *

from pymbe.query.metamodel_navigator import (is_type_undefined_mult,
                                    is_multiplicity_one,
                                    is_multiplicity_specific_finite,
                                    get_finite_multiplicity_types,
                                    identify_connectors_one_side,
                                    get_lower_multiplicity,
                                    get_upper_multiplicity,
                                    does_behavior_have_write_features,
                                    get_most_specific_feature_type,
                                    has_type_named,
                                    get_effective_lower_multiplicity,
                                    get_feature_bound_values,
                                    get_more_general_types,
                                    get_effective_basic_name)

from pymbe.metamodel import derive_inherited_featurememberships

from pymbe.text_concrete_syntax import serialize_kerml_atom

from pymbe.interpretation.working_maps import FeatureTypeWorkingMap
from pymbe.interpretation.execute_kerml_atoms import KermlForwardExecutor

from uuid import uuid4

## Key Helpers for the Algorithm

These helpers are yet to be implemented in the core of the Python tool and thus need to be more spelled out.

# Load up Kernel and Systems Libraries

Load up the model libraries into memory so that key features for subsetting can be found.

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "SystemsLibrary.json") as lib_data1:
    with lib_resources.path("pymbe.static_data", "KernelLibraryExpanded.json") as lib_data2:
            library_model = pm.Model.load_from_mult_post_files([lib_data1, lib_data2])

## Routines for Execution

The following sections are focused on solving the problem of mapping values to KerML types in the model. The approach taken here is to find one legal set of values for types in the model via an approach where the program will walk straight ahead in the model, deriving values as it goes. This approach is called "execution" here.

In [ ]:
def print_values_dictionary(model, values_dict):
    print_string = ""
    for k, v in values_dict.items():
        print_string = print_string + f">>>Key {model.get_element(k)} ({k}) has values {v}\n"
        
    print(print_string)

In [ ]:
def pretty_print_steps_log(builder_log):
    # helper to print the log in a way you can read it in the Jupyter notebook easily
    print("\n".join(builder_log))

## Atom Metadata Load

Bring up the Atom metadata.

filename = "A-2-Atoms"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

atoms_data = pm.Model.load_from_post_file(json_file)
atoms_data

## Simple Loop Trial

In [ ]:
filename = "UserModelTrialLoops"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "sysml_data" / filename

loop_data = pm.Model.load_from_post_file(json_file)
loop_data

loop_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in loop_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
execute_package = [ele for ele in packages if ele.declaredName == "LoopToExecute"][0]
execute_package

In [ ]:
trace_package = [ele for ele in packages if ele.declaredName == "LoopTrace"][0]
trace_package

In [ ]:
action_to_execute = [action for action in execute_package.throughOwningMembership
                     if get_effective_basic_name(action) == "Loop Master"][0]
action_to_execute

In [ ]:
occ_ns = [
        library_model_ns
        for library_model_ns in library_model.ownedElement
        if library_model_ns.throughOwningMembership[0].declaredName == "Occurrences"
    ][0]

In [ ]:
occ = [ele for ele in occ_ns.throughOwningMembership[0].throughOwningMembership
           if get_effective_basic_name(ele) == "Occurrence"][0]

time_slices = [ele for ele in occ.throughFeatureMembership if get_effective_basic_name(ele) == "timeSlices"][0]
time_slices

In [ ]:
def build_from_feature_slice_pattern(
    owner: Element,
    name: str,
    model: Model,
    specific_fields: dict[str, Any],
    feature_type: Element,
    direction: str = "",
    metatype: str = "Feature",
    connector_end: bool = False,
):

    """Creates a new element using a feature-style pattern that assumes:
    - The Feature will have some special kind of membership connecting it to the owner
    - The Feature may have a multiplicity
    - The Feature may have a type
    """
    typing_snippet = {}
    direction_snippet = {}
    member_kind = ""

    if feature_type is not None:
        typing_snippet = {"type": {"@id": feature_type}}

    if direction != "":
        direction_snippet = {"direction": direction}

    specific_fields = typing_snippet | direction_snippet

    feature_dict = create_element_data_dictionary(
        name=name, metaclass=metatype, model=model, specific_fields=specific_fields
    )

    feature_dict.update({"portionKind": "timeslice", "isPortion": True})

    new_ele = Element.new(data=feature_dict, model=model)

    # TODO: Add more cases here
    if (
        metatype in {"Feature", "Connector", "Succession", "Step"}
        or "Usage" in metatype
    ):
        if connector_end:
            member_kind = "EndFeatureMembership"
        else:
            member_kind = "FeatureMembership"

    new_element_ownership_pattern(
        owner=owner, ele=new_ele, model=model, member_kind=member_kind
    )

    # need to subset the library timeSlices

    build_from_binary_relationship_pattern(
        source=new_ele,
        target=time_slices,
        model=model,
        metatype="Subsetting",
        owned_by_source=True,
        owns_target=False,
        alternative_owner=None,
        specific_fields={},
    )

    if feature_type is not None:
        build_from_binary_relationship_pattern(
            source=new_ele,
            target=feature_type,
            model=model,
            metatype="FeatureTyping",
            owned_by_source=True,
            owns_target=False,
            alternative_owner=None,
            specific_fields={},
        )

    return new_ele

In [ ]:
class TimeSliceWorkingMap():
    # The working dictionary for atoms to atoms (objects)
    _working_dict: dict[str, dict[str, list[Element]]] = field(default_factory=dict)

    def __init__():
        self._working_dict = {}

In [ ]:
class ForwardSysMLExecution():

    working_slices: List[Element] = field(default_factory=list)

    execution_action: Element

    # a log that is indexed by instance that walks through the executor steps taken
    _builder_log = {}

    # a log that walks thorugh the steps of traversal between elements
    _traversal_log: list[str]

    # a log that records the state of the value assignments at particular points
    _value_map_log = []

    # Prevent infinite loops
    _safety_counter: int
    
    def __init__(self):
        self.working_slices = {}
        self.current_frame_list = {}
        self.working_slice_activation = {}
        self._builder_log = {}
        self._traversal_log = []
        self._value_map_log = []
        self.execution_action = None
        self._safety_counter = 0

    def start_execution(self, target_package: Element, execution_name: str):
        """
        Create the outer action that will carry the execution trace.
        """
        self.execution_action = build_from_classifier_pattern(
            owner=target_package,
            name=execution_name,
            model=target_package._model,
            metatype="ActionDefinition",
            superclasses=[],
            specific_fields={},
        )
    
    def start_behavior(self, behavior_element: Element, new_owner: Element):
        """
        Start the behavior by adding an initial time slice for it to the working stack
        """

        # check that we have a SuccessionAsUsage that points to the start node

        successions = [ele for ele in behavior_element.throughFeatureMembership
                       if ele._metatype == "SuccessionAsUsage"]

        start_succession = [ele for ele in successions
                            if get_effective_basic_name(ele.source[0]) == "start"][0]

        if new_owner is None:
            new_slice = build_from_feature_slice_pattern(
                owner=self.execution_action,
                name=f"\"{get_effective_basic_name(behavior_element)} Top Frame\"",
                model=behavior_element._model,
                specific_fields={},
                feature_type=None,
                direction = "",
                metatype = "OccurrenceUsage",
                connector_end = False,
            )

            self.current_frame_list.update({'Core Thread': 1})

            new_working_slice = build_from_feature_slice_pattern(
                owner=new_slice,
                name="Core Thread" + \
                    f" Frame {self.current_frame_list['Core Thread']}\"",
                model=behavior_element._model,
                specific_fields={},
                feature_type=None,
                direction = "",
                metatype = "OccurrenceUsage",
                connector_end = False,
            )

            self.working_slice_activation.update({new_working_slice: start_succession.target[0]})

            self.working_slices.update({'Core Thread': new_working_slice})
    
    def consider_current_step(self):
        """
        Cycle through the current working slices to see if there is a set of steps 
        inside the step to take.
        """

        for thread_name, working_slice in self.working_slices.items():

            original_step = self.working_slice_activation[working_slice]

            print(f"Considering {original_step} within {working_slice}")

            internal_steps = False

            for feature in original_step.throughFeatureMembership:
                if feature._metatype in ("ActionUsage", "AssignActionUsage"):
                    print(f"Found subsetep {feature} within {working_slice}")

                    internal_steps = True

            if not internal_steps:
                self.consider_next_step()
    
    def consider_next_step(self):
        """
        Cycle through the current working slices to see if there is yet a new step 
        to take. Eventually order next steps by time.
        """

        for thread_name, working_slice in self.working_slices.items():

            original_step = self.working_slice_activation[working_slice]

            for next_step in original_step.throughSuccessionAsUsage:
                print(f"Found next step as {next_step}")

    def record_slice(self):
        """
        Record the new time slices ... this should probably move to the working structure.
        """

In [ ]:
executor = ForwardSysMLExecution()

In [ ]:
executor.start_execution(target_package=trace_package, execution_name="Loop Trial")

In [ ]:
executor.start_behavior(behavior_element=action_to_execute, new_owner=None)

In [ ]:
executor.consider_current_step()

In [ ]:
executor.current_frame_list

In [ ]:
trace_package.throughOwningMembership

In [ ]:
trace_package.throughOwningMembership[0].throughFeatureMembership[0]

In [ ]:
actions = [ele for ele in loop_data.elements.values() if ele._metatype == 'ActionUsage']
actions

In [ ]:
features = action_to_execute.throughFeatureMembership
features

In [ ]:
[ele for ele in action_to_execute.throughFeatureMembership
     if hasattr(ele, "throughSuccessionAsUsage") or hasattr(ele, "throughTransitionUsage")]